In [1]:
import requests
from bs4 import BeautifulSoup
from zipfile import ZipFile
import xml.etree.ElementTree as ET
from pathlib import Path
from pykml import parser
from fastkml import kml
import pandas as pd
import lxml.etree as et
from io import StringIO
from html.parser import HTMLParser

In [2]:
def webscrap():

    #URL = "https://buzzardsbay.org/enjoy-buzzards-bay/beach-information/"
    #page = requests.get(URL)
    #print(page.text)

    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
    }
    URL = "https://buzzardsbay.org/enjoy-buzzards-bay/beach-information/"
    page = requests.get(URL, headers=headers)

    soup = BeautifulSoup(page.content, "html.parser")
    map_class = soup.find_all("div", class_="flxmap-container")
    #print(soup)
    map_id = soup.find(id="flxmap-617f4b62e1e1d")
    #print(map_id)
    #print(requests.get("https:\/\/buzzardsbay.org\/gis-data\/beaches_buzzards_bay_pts_2011.kmz", headers=headers))

    filename = '/Users/gupsb/Downloads/beaches_buzzards_bay_pts_2011.zip'

    kmz = ZipFile(filename, 'r')
    kml_file = Path(r'/Users/gupsb/Downloads/beaches_buzzards_bay_pts_2011/doc.kml')
    tree = ET.parse(kml_file)
    root = tree.getroot()
    with open(kml_file) as f:
        folder = parser.parse(f).getroot().Document.Folder

    plnm = []
    cordi = []
    use_category = []
    ownership = []
    town =[]
    acres = []
    check = True
    for pm in folder.Placemark:
        plnm1 = pm.name
        plcs1 = pm.Point.coordinates
        if check:
            desc_old = pm.description.text
            #print(desc_old)
            #for tables in et.HTML(desc_old).find("body/table/tr/td/table"):
                #print(tables)
            table = et.HTML(desc_old).find("body/table/tr/td/table")
            #print(table)
            rows = iter(table)
            headers = [col.text for col in next(rows)]
            #print(headers)
            for row in rows:
                values = [col.text for col in row]
                if values[0] == 'OWNERSHIP':
                    ownership.append(values[1])
                elif values[0] == 'TOWN':
                    town.append(values[1])
                elif values[0] == 'USE_CATEGO':
                    use_category.append(values[1])
                elif values[0] == 'ACRES':
                        acres.append(values[1])

                
                #print (dict(zip(headers, values)))
           
        plnm.append(plnm1.text)
        cordi.append(plcs1.text)
        #meta_element=desc_old_tree_html.findall("head")[0].findall("Meta")[0]
    #print(plnm)
    #print(cordi)
    #print(access)
    db=pd.DataFrame()
    db['Beach']=plnm
    db['Coordinates']=cordi
    db['Town']=town
    db['Ownership']=ownership
    db['Accessibilty']=use_category
    db['Area']=acres
    
    return db
    

In [3]:
data = webscrap()

In [4]:
data

Beach                              Coordinates  \
0                  Town Beach   -70.80859986052559,41.65830356813273,0   
1           Fort Phoenix Town   -70.90262689513151,41.62419557821986,0   
2             Horseneck Beach   -71.04870470650639,41.50275851563166,0   
3   Demarest Lloyd State Park     -70.98025037657065,41.527661939068,0   
4                 Jones Beach   -70.93097682532019,41.60506978809762,0   
..                        ...                                      ...   
93            Wood Neck River    -70.63667983103267,41.5756967307811,0   
94       Hamlin's Point Beach   -70.64861873569222,41.56948654160055,0   
95               Racing Beach   -70.65418479760005,41.55400014874505,0   
96                   The Knob   -70.65851179684259,41.54317779866575,0   
97    Saconessett Hills Beach   -70.64115995871666,41.57786181554928,0   

            Town    Ownership Accessibilty    Area  
0   Mattapoisett    Municipal    Municipal   0.944  
1      Fairhaven    Municipal    Municipal   1.241  
2       Westport        State        State  39.461  
3      Dartmouth        State        State   9.056  
4      Dartmouth    Municipal    Municipal   2.044  
..           ...          ...          ...     ...  
93      Falmouth    Municipal    Municipal   5.832  
94      Falmouth      Private  Semi-Public   2.223  
95      Falmouth  Association  Semi-Public   3.408  
96      Falmouth      Private  Semi-Public   1.075  
97      Falmouth  Association  Semi-Public   1.105  

[98 rows x 6 columns]